In [30]:
import cv2
import os
import numpy as np

dir = "./smaller-data"
for filename in os.listdir(dir):
    filepath = (os.path.join(dir, filename))
    name = filename.split(".")
    name = name[0]

    videoFile = np.load(filepath)
    colorImages = videoFile['colorImages']
    boundingBox = videoFile['boundingBox']
    landmarks2D = videoFile['landmarks2D']

    height = colorImages.shape[0]
    width = colorImages.shape[1]
    for i in range(colorImages.shape[-1]):
        viss = []
        img = np.zeros((height, width, 3), np.uint8)
        img1 = colorImages[:, :, :, i]
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

        bb = boundingBox[:, :, i]
        for mark in landmarks2D[:, :, i]:
            cv2.circle(img, (int(mark[0]), int(mark[1])), 1, (255, 255, 255))

        bbox_true = [int(bb[0, 0]), int(bb[0, 1]), int(bb[3, 0]), int(bb[3, 1])]
        cv2.rectangle(img,
                        (int(bb[0, 0]), int(bb[0, 1])), 
                        (int(bb[3, 0]), int(bb[3, 1])),
                        (0, 0, 255), 1)

        combine = np.hstack((img, img1))
        cv2.imshow('video', combine)
        c = cv2.waitKey(1)
        if c == 27:
            break
    cv2.destroyAllWindows()

Error opening video stream or file
0
